In [ ]:
import pandas as pd
import numpy as np
import re
from re import sub
import multiprocessing

from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec
from gensim.test.utils import get_tmpfile
from gensim.models import KeyedVectors


from time import time 
from collections import defaultdict


In [ ]:
df=pd.read_csv("../../../jigsaw-toxic-severity-rating/comments_to_score.csv")


In [ ]:
df


,comment_id,text
0,114890,"""\n \n\nGjalexei, you asked about whether ther..."
1,732895,"Looks like be have an abuser , can you please ..."
2,1139051,I confess to having complete (and apparently b...
3,1434512,"""\n\nFreud's ideas are certainly much discusse..."
4,2084821,It is not just you. This is a laundry list of ...
...,...,...
7532,504235362,"Go away, you annoying vandal."
7533,504235566,This user is a vandal.
7534,504308177,""" \n\nSorry to sound like a pain, but one by f..."
7535,504570375,Well it's pretty fucking irrelevant now I'm un...


In [ ]:
from nltk.corpus import stopwords
import re
def text_cleaning(text):
    forbidden_words = set(stopwords.words('english'))
    if text:
        text = ' '.join(text.split('.'))
        text = re.sub('\/',' ',text)
        text = re.sub(r'\\',' ',text)
        text = re.sub(r'((http)\S+)','',text)
        text = re.sub(r'\s+', ' ', re.sub('[^A-Za-z]', ' ', text.strip().lower())).strip()
        text = re.sub(r'\W+', ' ', text.strip().lower()).strip()
        text = [word for word in text.split() if word not in forbidden_words]
        return text
    return []


In [ ]:
from nltk.tokenize import sent_tokenize,word_tokenize
from nltk.corpus import stopwords
from string import punctuation
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()


In [ ]:
def lemmatizing(s):
    l=s.split(" ")
    l2=[lemmatizer.lemmatize(word) for word in l]
    s=" ".join(l2)
    return s


In [ ]:
df.text = df.text.apply(lambda x: ' '.join(text_cleaning(x)))


In [ ]:
df.text = df.text.apply(lambda x: lemmatizing(x))


In [ ]:
df


,comment_id,text
0,114890,gjalexei asked whether anti editorializing pol...
1,732895,look like abuser please look thanks
2,1139051,confess complete apparently blissful ignorance...
3,1434512,freud idea certainly much discussed today woul...
4,2084821,laundry list stupid allegation scooped god kno...
...,...,...
7532,504235362,go away annoying vandal
7533,504235566,user vandal
7534,504308177,sorry sound like pain one following tad stalki...
7535,504570375,well pretty fucking irrelevant unblocked aint


In [ ]:
sent = [row for row in df.text]
phrases = Phrases(sent, min_count=1, progress_per=50000)
bigram = Phraser(phrases)
sentences = bigram[sent]
sentences[1]


'look like abuser please look thanks'

In [ ]:
sentences = [word_tokenize(sent) for sent in df.text]


In [ ]:
sentences[1]


['look', 'like', 'abuser', 'please', 'look', 'thanks']

In [ ]:
w2v_model = Word2Vec(min_count=3,
                     window=4,
                     vector_size=300,
                     sample=1e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=multiprocessing.cpu_count()-1)

start = time()

w2v_model.build_vocab(sentences, progress_per=50000)

print('Time to build vocab: {} mins'.format(round((time() - start) / 60, 2)))


Time to build vocab: 0.0 mins


In [ ]:
start = time()

w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)

print('Time to train the model: {} mins'.format(round((time() - start) / 60, 2)))

w2v_model.init_sims(replace=True)


Time to train the model: 0.06 mins


C:\Users\Hector\AppData\Local\Temp/ipykernel_10776/397523926.py:7: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  w2v_model.init_sims(replace=True)


In [ ]:
w2v_model.save("word2vec.model")


In [ ]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from sklearn.cluster import KMeans


In [ ]:
word_vectors = Word2Vec.load("word2vec.model").wv


In [ ]:
word_vectors


In [ ]:
model = KMeans(n_clusters=2, max_iter=1000, random_state=True, n_init=50).fit(X=word_vectors.vectors.astype('double'))


In [ ]:
word_vectors.similar_by_vector(model.cluster_centers_[0], topn=10, restrict_vocab=None)


[('loser', 0.9875553250312805),
 ('ya', 0.9852948784828186),
 ('motherfucker', 0.9850976467132568),
 ('basement', 0.9843053817749023),
 ('bout', 0.9835658669471741),
 ('son', 0.9834238290786743),
 ('butt', 0.9832683801651001),
 ('got', 0.9830428957939148),
 ('nerd', 0.9826918244361877),
 ('fagget', 0.9823668599128723)]

In [ ]:
word_vectors.similar_by_vector(model.cluster_centers_[1], topn=10, restrict_vocab=None)


[('cranky', 0.9998353123664856),
 ('banner', 0.9998348951339722),
 ('enlightenment', 0.9998064637184143),
 ('winter', 0.9998010396957397),
 ('empirically', 0.9998007416725159),
 ('distinguished', 0.9997978210449219),
 ('strive', 0.9997966885566711),
 ('exempt', 0.9997962713241577),
 ('atom', 0.9997950196266174),
 ('newer', 0.9997943639755249)]

In [ ]:
positive_cluster_index = 1
positive_cluster_center = model.cluster_centers_[positive_cluster_index]
negative_cluster_center = model.cluster_centers_[1-positive_cluster_index]


In [ ]:
words = pd.DataFrame(word_vectors.index_to_key)


In [ ]:
words


,0
0,wikipedia
1,article
2,page
3,fuck
4,like
...,...
9133,portrait
9134,taliban
9135,reptile
9136,enacted


In [ ]:
words.columns = ['words']
words['vectors'] = words.words.apply(lambda x: word_vectors[f'{x}'])
words['cluster'] = words.vectors.apply(lambda x: model.predict([np.array(x)]))
words.cluster = words.cluster.apply(lambda x: x[0])


In [ ]:
words['cluster_value'] = [1 if i==positive_cluster_index else -1 for i in words.cluster]
words['closeness_score'] = words.apply(lambda x: 1/(model.transform([x.vectors]).min()), axis=1)
words['sentiment_coeff'] = words.closeness_score * words.cluster_value


In [ ]:
words


,words,vectors,cluster,cluster_value,closeness_score,sentiment_coeff
0,wikipedia,"[-0.07521146, 0.026654694, -0.060870226, 0.109...",0,-1,1.285930,-1.285930
1,article,"[0.0349816, 0.08546219, -0.041096292, -0.06351...",1,1,2.410108,2.410108
2,page,"[0.069779545, 0.07167754, -0.11654684, -0.0157...",1,1,1.743097,1.743097
3,fuck,"[-0.05979239, 0.016655028, -0.13145107, 0.0855...",0,-1,1.600695,-1.600695
4,like,"[-0.01731605, 0.0014586718, -0.0898409, -0.003...",0,-1,1.447705,-1.447705
...,...,...,...,...,...,...
9133,portrait,"[-0.009802352, 0.08417154, -0.0927693, -0.0251...",1,1,28.548704,28.548704
9134,taliban,"[-0.006065281, 0.08411236, -0.09350408, -0.030...",1,1,26.365268,26.365268
9135,reptile,"[-0.0037926005, 0.08505091, -0.09321178, -0.02...",1,1,26.783980,26.783980
9136,enacted,"[0.0013086041, 0.090937935, -0.09649347, -0.03...",1,1,16.111376,16.111376


In [ ]:
sentiment_map = words
sentiment_dict = dict(zip(sentiment_map.words.values, sentiment_map.sentiment_coeff.values))


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from IPython.display import display


In [ ]:
tfidf = TfidfVectorizer(tokenizer=lambda x: x.split(), norm=None)
tfidf.fit(df.text)
features = pd.Series(tfidf.get_feature_names())
transformed = tfidf.transform(df.text)


C:\Users\Hector\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:516: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
C:\Users\Hector\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
def create_tfidf_dictionary(x, transformed_file, features):
    '''
    create dictionary for each input sentence x, where each word has assigned its tfidf score
    
    inspired  by function from this wonderful article: 
    https://medium.com/analytics-vidhya/automated-keyword-extraction-from-articles-using-nlp-bfd864f41b34
    
    x - row of dataframe, containing sentences, and their indexes,
    transformed_file - all sentences transformed with TfidfVectorizer
    features - names of all words in corpus used in TfidfVectorizer

    '''
    vector_coo = transformed_file[x.name].tocoo()
    vector_coo.col = features.iloc[vector_coo.col].values
    dict_from_coo = dict(zip(vector_coo.col, vector_coo.data))
    return dict_from_coo

def replace_tfidf_words(x, transformed_file, features):
    '''
    replacing each word with it's calculated tfidf dictionary with scores of each word
    x - row of dataframe, containing sentences, and their indexes,
    transformed_file - all sentences transformed with TfidfVectorizer
    features - names of all words in corpus used in TfidfVectorizer
    '''
    dictionary = create_tfidf_dictionary(x, transformed_file, features)   
    return list(map(lambda y:dictionary[f'{y}'], x.text.split()))


In [ ]:
%%time
replaced_tfidf_scores = df.apply(lambda x: replace_tfidf_words(x, transformed, features), axis=1)


Wall time: 1.83 s


In [ ]:
replaced_tfidf_scores


0       [9.234564993267135, 5.427902503496815, 5.27375...
1       [7.827994035568558, 2.7484042043230463, 8.3182...
2       [8.318274261392979, 5.637252732678689, 5.54568...
3       [65.0876216367922, 9.860999800125931, 5.496895...
4       [8.829099885158971, 5.052514850625929, 4.10171...
                              ...                        
7532    [3.3723552820164158, 4.723705486750285, 6.2141...
7533               [3.507717245679938, 4.852538358593254]
7534    [4.649597514596564, 5.520992926562828, 2.74840...
7535    [3.6586158901208194, 5.363363982359244, 3.9539...
7536    [6.077564572117022, 4.134698565442937, 4.91707...
Length: 7537, dtype: object

In [ ]:
def replace_sentiment_words(word, sentiment_dict):
    '''
    replacing each word with its associated sentiment score from sentiment dict
    '''
    try:
        out = sentiment_dict[word]
    except KeyError:
        out = 0
    return out


In [ ]:
replaced_closeness_scores = df.text.apply(lambda x: list(map(lambda y: replace_sentiment_words(y, sentiment_dict), x.split())))


In [ ]:
replaced_closeness_scores


0       [0, 9.054821496294519, 6.969735481561163, 6.84...
1       [9.635027787497972, -1.4477052241157573, 32.52...
2       [8.773734007549283, 8.172535263059862, 9.25261...
3       [8.68947270133028, 13.186975662811886, 12.1701...
4       [0, 8.929117151265888, 3.5818176441818577, 11....
                              ...                        
7532    [-1.2820707884968412, -1.4732873605101704, 10....
7533             [-1.2078422525767494, 7.303036891570346]
7534    [6.229887655607716, 15.855141848368733, -1.447...
7535    [8.983524868692394, 18.542841544031457, -1.293...
7536    [16.0161697085287, 2.1482053409843753, 12.3392...
Name: text, Length: 7537, dtype: object

In [ ]:
df


,comment_id,text
0,114890,gjalexei asked whether anti editorializing pol...
1,732895,look like abuser please look thanks
2,1139051,confess complete apparently blissful ignorance...
3,1434512,freud idea certainly much discussed today woul...
4,2084821,laundry list stupid allegation scooped god kno...
...,...,...
7532,504235362,go away annoying vandal
7533,504235566,user vandal
7534,504308177,sorry sound like pain one following tad stalki...
7535,504570375,well pretty fucking irrelevant unblocked aint


In [ ]:
replacement_df = pd.DataFrame(data=[replaced_closeness_scores, replaced_tfidf_scores, df.text]).T
replacement_df.columns = ['sentiment_coeff', 'tfidf_scores', 'sentence']
replacement_df['sentiment_rate'] = replacement_df.apply(lambda x: np.array(x.loc['sentiment_coeff']) @ np.array(x.loc['tfidf_scores']), axis=1)


In [ ]:
replacement_df


,sentiment_coeff,tfidf_scores,sentence,sentiment_rate
0,"[0, 9.054821496294519, 6.969735481561163, 6.84...","[9.234564993267135, 5.427902503496815, 5.27375...",gjalexei asked whether anti editorializing pol...,3711.514078
1,"[9.635027787497972, -1.4477052241157573, 32.52...","[7.827994035568558, 2.7484042043230463, 8.3182...",look like abuser please look thanks,423.319849
2,"[8.773734007549283, 8.172535263059862, 9.25261...","[8.318274261392979, 5.637252732678689, 5.54568...",confess complete apparently blissful ignorance...,2337.141664
3,"[8.68947270133028, 13.186975662811886, 12.1701...","[65.0876216367922, 9.860999800125931, 5.496895...",freud idea certainly much discussed today woul...,19033.450204
4,"[0, 8.929117151265888, 3.5818176441818577, 11....","[8.829099885158971, 5.052514850625929, 4.10171...",laundry list stupid allegation scooped god kno...,474.067310
...,...,...,...,...
7532,"[-1.2820707884968412, -1.4732873605101704, 10....","[3.3723552820164158, 4.723705486750285, 6.2141...",go away annoying vandal,89.359091
7533,"[-1.2078422525767494, 7.303036891570346]","[3.507717245679938, 4.852538358593254]",user vandal,31.201498
7534,"[6.229887655607716, 15.855141848368733, -1.447...","[4.649597514596564, 5.520992926562828, 2.74840...",sorry sound like pain one following tad stalki...,3920.360229
7535,"[8.983524868692394, 18.542841544031457, -1.293...","[3.6586158901208194, 5.363363982359244, 3.9539...",well pretty fucking irrelevant unblocked aint,345.895976


In [ ]:
replacement_df["score"]=[sum(i) for i in replacement_df.sentiment_coeff]

In [ ]:
replacement_df

,sentiment_coeff,tfidf_scores,sentence,sentiment_rate,score
0,"[0, 9.054821496294519, 6.969735481561163, 6.84...","[9.234564993267135, 5.427902503496815, 5.27375...",gjalexei asked whether anti editorializing pol...,3711.514078,516.694056
1,"[9.635027787497972, -1.4477052241157573, 32.52...","[7.827994035568558, 2.7484042043230463, 8.3182...",look like abuser please look thanks,423.319849,52.631102
2,"[8.773734007549283, 8.172535263059862, 9.25261...","[8.318274261392979, 5.637252732678689, 5.54568...",confess complete apparently blissful ignorance...,2337.141664,351.770480
3,"[8.68947270133028, 13.186975662811886, 12.1701...","[65.0876216367922, 9.860999800125931, 5.496895...",freud idea certainly much discussed today woul...,19033.450204,1684.577946
4,"[0, 8.929117151265888, 3.5818176441818577, 11....","[8.829099885158971, 5.052514850625929, 4.10171...",laundry list stupid allegation scooped god kno...,474.067310,91.164539
...,...,...,...,...,...
7532,"[-1.2820707884968412, -1.4732873605101704, 10....","[3.3723552820164158, 4.723705486750285, 6.2141...",go away annoying vandal,89.359091,15.040490
7533,"[-1.2078422525767494, 7.303036891570346]","[3.507717245679938, 4.852538358593254]",user vandal,31.201498,6.095195
7534,"[6.229887655607716, 15.855141848368733, -1.447...","[4.649597514596564, 5.520992926562828, 2.74840...",sorry sound like pain one following tad stalki...,3920.360229,529.149233
7535,"[8.983524868692394, 18.542841544031457, -1.293...","[3.6586158901208194, 5.363363982359244, 3.9539...",well pretty fucking irrelevant unblocked aint,345.895976,60.345045


In [ ]:
validation_data = pd.read_csv("../../../jigsaw-toxic-severity-rating/validation_data.csv")


In [ ]:
validation_data


,worker,less_toxic,more_toxic
0,313,This article sucks \n\nwoo woo wooooooo,WHAT!!!!!!!!?!?!!?!?!!?!?!?!?!!!!!!!!!!!!!!!!!...
1,188,"""And yes, people should recognize that but the...",Daphne Guinness \n\nTop of the mornin' my fav...
2,82,"Western Media?\n\nYup, because every crime in...","""Atom you don't believe actual photos of mastu..."
3,347,And you removed it! You numbskull! I don't car...,You seem to have sand in your vagina.\n\nMight...
4,539,smelly vagina \n\nBluerasberry why don't you ...,"hey \n\nway to support nazis, you racist"
...,...,...,...
30103,461,I'm sorry. I'm not an admin. I will give you t...,get out my large penis
30104,527,I'm sorry. I'm not an admin. I will give you t...,get out my large penis
30105,352,"wow...\nare you out of your mind, how was my e...",Piss off you slant eyed-gook
30106,311,"wow...\nare you out of your mind, how was my e...",Piss off you slant eyed-gook


In [ ]:
validation_data.less_toxic = validation_data.less_toxic.apply(lambda x: ' '.join(text_cleaning(x)))
validation_data.more_toxic = validation_data.more_toxic.apply(lambda x: ' '.join(text_cleaning(x)))


In [ ]:
validation_data.less_toxic = validation_data.less_toxic.apply(lambda x: lemmatizing(x))
validation_data.more_toxic = validation_data.more_toxic.apply(lambda x: lemmatizing(x))


In [ ]:
validation_data


,worker,less_toxic,more_toxic
0,313,article suck woo woo wooooooo,wher sexy pic gone main page put back
1,188,yes people recognize usually one first objecti...,daphne guinness top mornin favourite alien lif...
2,82,western medium yup every crime entire world ca...,atom believe actual photo masturbation sexuall...
3,347,removed numbskull care say anymore life go ahe...,seem sand vagina might want wash stinking hole...
4,539,smelly vagina bluerasberry model cheesy blue v...,hey way support nazi racist
...,...,...,...
30103,461,sorry admin give three piece advice though fir...,get large penis
30104,527,sorry admin give three piece advice though fir...,get large penis
30105,352,wow mind edit talk page vandalism simply expla...,piss slant eyed gook
30106,311,wow mind edit talk page vandalism simply expla...,piss slant eyed gook


In [ ]:
replacement_df["sentence"]


0       gjalexei asked whether anti editorializing pol...
1                     look like abuser please look thanks
2       confess complete apparently blissful ignorance...
3       freud idea certainly much discussed today woul...
4       laundry list stupid allegation scooped god kno...
                              ...                        
7532                              go away annoying vandal
7533                                          user vandal
7534    sorry sound like pain one following tad stalki...
7535        well pretty fucking irrelevant unblocked aint
7536    team name great britain northern ireland blind...
Name: sentence, Length: 7537, dtype: object

In [ ]:
float(replacement_df.loc[replacement_df["sentence"] == "article suck woo woo wooooooo", 'sentiment_rate'])


967.7832400384474

In [ ]:
less_toxic_score=[]
for i in validation_data["less_toxic"]:
    less_toxic_score.append(replacement_df.loc[replacement_df["sentence"] == i, 'sentiment_rate'])
            

In [ ]:
more_toxic_score=[]
for i in validation_data["more_toxic"]:
    more_toxic_score.append(replacement_df.loc[replacement_df["sentence"] == i, 'sentiment_rate'])
    

In [ ]:
for i in range(len(less_toxic_score)):
    if len(less_toxic_score[i])==0:
        less_toxic_score[i]=0.0
    else:
        less_toxic_score[i]=less_toxic_score[i].iloc[0]
        

In [ ]:
for i in range(len(more_toxic_score)):
    if len(more_toxic_score[i])==0:
        more_toxic_score[i]=0.0
    else:
        more_toxic_score[i]=more_toxic_score[i].iloc[0]
        

In [ ]:
validation_data["less_toxic_score"]=less_toxic_score
validation_data["more_toxic_score"]=more_toxic_score


In [ ]:
scores=[]
for index, row in validation_data.iterrows():
    if row["less_toxic_score"]<row["more_toxic_score"]:
        scores.append(1)
    else:
        scores.append(0)
        

In [ ]:
validation_data["scores"]=scores


In [ ]:
validation_data


,worker,less_toxic,more_toxic,less_toxic_score,more_toxic_score,scores,less_toxic_score2,more_toxic_score2,scores2
0,313,article suck woo woo wooooooo,wher sexy pic gone main page put back,967.783240,430.683574,0,55.580534,430.683574,0
1,188,yes people recognize usually one first objecti...,daphne guinness top mornin favourite alien lif...,1941.125082,999.723686,0,250.942776,999.723686,0
2,82,western medium yup every crime entire world ca...,atom believe actual photo masturbation sexuall...,867.914583,5276.603121,1,132.260623,5276.603121,0
3,347,removed numbskull care say anymore life go ahe...,seem sand vagina might want wash stinking hole...,289.345313,526.939685,1,59.369599,526.939685,0
4,539,smelly vagina bluerasberry model cheesy blue v...,hey way support nazi racist,1025.813672,108.848273,0,127.003671,108.848273,1
...,...,...,...,...,...,...,...,...,...
30103,461,sorry admin give three piece advice though fir...,get large penis,5688.393691,79.340981,0,749.258559,79.340981,1
30104,527,sorry admin give three piece advice though fir...,get large penis,5688.393691,79.340981,0,749.258559,79.340981,1
30105,352,wow mind edit talk page vandalism simply expla...,piss slant eyed gook,0.000000,283.727159,1,0.000000,283.727159,0
30106,311,wow mind edit talk page vandalism simply expla...,piss slant eyed gook,0.000000,283.727159,1,0.000000,283.727159,0


In [ ]:
validation_data["scores"].value_counts(normalize=True)


0    0.63239
1    0.36761
Name: scores, dtype: float64